## Adequação do banco de dados ao Heroku

Para usar o plano gratuito (_Hob Dev_) do addon de Postgres no Heroku, há um [limite de 10000 linhas](https://elements.heroku.com/addons/heroku-postgresql#pricing) para o banco de dados. Infelizmente, no formato atual, algumas tabelas são maiores do que isso:

In [1]:
import pandas as pd
from sqlalchemy import create_engine
DATABASE_URL = "sqlite:///../data/trades.db"
engine = create_engine(DATABASE_URL)

tables = ['import', 'export', 'ncm', 'uf', 'top_by_state_and_month',
          'top_by_state_and_year', 'state_contributions']
dfs=[]
for table in tables:
    df = pd.read_sql(f'SELECT "{table}" "table", COUNT(*) n_rows FROM {table}', engine)
    dfs.append(df)
display(pd.concat(dfs).set_index('table'))

,n_rows
table,
import,5594885
export,3681931
ncm,13117
uf,34
top_by_state_and_month,2016
top_by_state_and_year,514
state_contributions,56


No caso específico das tabelas `import` e `export`, o limite é excedido mesmo quando se considera cada ano individualmente:

In [2]:
pd.read_sql(f'SELECT strftime(\"%Y\", DATA) year, COUNT(*) FROM import GROUP BY year', engine)

,year,COUNT(*)
0,2017,1823961
1,2018,1866511
2,2019,1904413


In [3]:
pd.read_sql(f'SELECT strftime(\"%Y\", DATA) year, COUNT(*) FROM export GROUP BY year', engine)

,year,COUNT(*)
0,2017,1096765
1,2018,1168283
2,2019,1416883


Assim, não seria viável armazenar a maioria dos dados em sua forma inicial.

Já as tabelas de estatísticas (`top_by_state_and_month`, `top_by_state_and_year` e `state_contributions`) têm um tamanho aceitável. No entanto, o dashboard faz um `JOIN` das duas primeiras com a tabela `ncm`, que é grande. Sendo assim, uma alternativa seria manter uma versão reduzida da tabela `ncm`, que incluísse apenas os produtos que aparecem nas tabelas de estatísticas. Afinal, eles são bem poucos, e a maioria aparece tanto em `top_by_state_and_month` quanto em `top_by_state_and_year`:

In [4]:
pd.read_sql(f'SELECT COUNT(DISTINCT(product)) FROM top_by_state_and_month', engine)

,COUNT(DISTINCT(product))
0,282


In [5]:
pd.read_sql(f'SELECT COUNT(DISTINCT(product)) FROM top_by_state_and_year', engine)

,COUNT(DISTINCT(product))
0,138


In [6]:
query = '''
SELECT *
FROM ncm
WHERE CO_NCM IN
    (SELECT DISTINCT(product)
     FROM top_by_state_and_year
     UNION SELECT DISTINCT(product)
     FROM top_by_state_and_month)
'''
pd.read_sql(query, engine)

,CO_NCM,CO_UNID,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING
0,1012100,11,10121,1990,1990,1,15,110,9000,142,8099,Cavalos reprodutores de raça pura,"Caballos de pura raza, para procreación",Pure-bred horses
1,2013000,10,20130,1100,1100,1,1112,324,4000,1010,501,"Carnes desossadas de bovino, frescas ou refrig...","Carnes deshuesadas de la esp.bovina, frescas o...","Fresh or chilled bovine meat, boneless"
2,2023000,10,20230,1100,1100,1,1122,324,4000,1010,501,"Carnes desossadas de bovino, congeladas","Carnes deshuesadas de la especie bovina, conge...","Frozen bovine meat, boneless"
3,2032900,10,20329,1117,1990,1,1222,220,4000,1010,507,"Outras carnes de suíno, congeladas","Las demás carnes de la esp.porcina, congeladas","Other swine meat, frozen"
4,2062990,10,20629,1511,1269,1,1252,324,4000,1010,599,"Outras miudezas comestíveis de bovino, congeladas","Los dem.despojos comestibles de la esp.bovina,...",Other frozen edible bovine offal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,90039090,10,900390,3990,3043,3,88422,240,4000,3250,2600,Outras partes para armações de óculos e artigo...,Las dem.partes p/monturas de gafas y artículos...,Other parts for frames of glasses and similar ...
294,90318099,11,903180,3715,3715,3,87425,110,1000,2651,2600,"Outros instrumentos, aparelhos e máquinas de m...","Los dem.instrumentos, aparatos y máqs.p/medida...",Oth.instr.applian.and mach.of measuring/checking
295,94069020,10,940690,3236,3236,3,8110,240,3000,2395,2500,Construções pré-fabricadas com estrutura de fe...,Construcciones prefabricadas con paredes de hi...,Prefabricated buildings with iron or steel str...
296,97030000,11,970300,3990,3990,3,8963,311,9000,9000,8099,Produções originais de arte estatuária ou de e...,Obras originales de estatuaria o escultura,Original sculptures and statuary
